In [1]:
import rpy2.rinterface

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

In [7]:
%%R
library(ggplot2)
library(zoo)
library(dplyr)
library(data.table)

source("/Users/grlurton/Documents/hivdr/patient_data.R")
data_dir <- "/Users/grlurton/data/dhis/rdc/hivdr/"
metadata_dir <- "/Users/grlurton/data/dhis/rdc/hivdr/metadata/"

load_metadata(metadata_dir)

cordaid <- readRDS(paste0(data_dir, 'Cordaid_TRAITEMENTS.rds'))
cordaid <- merge(cordaid, M_org_units, by.x = 'orgUnit', by.y='id', all.y = FALSE)
pnls <- readRDS(paste0(data_dir, 'PNLS.rds'))
pnls <- merge(pnls, M_org_units, by.x = 'orgUnit', by.y='id', all.y = FALSE)

pnls_id <- 'Dd2G5zI0o0a'
cordaid_id <- 'Yj8caUQs178'

ancient_cat_option <-  c('vZ6Os4BJvum','ggod3chlUCG')
ancient_cat_combo <- M_category_combos$CatComboOpt_id[M_category_combos$CatOpt_id.1 %in% ancient_cat_option]

In [165]:
%%R
colnames(ds3)

[1] "level_2_name" "level_2_id"   "level_3_name" "level_3_id"   "source"      
[6] "periods"      "value"       


## CSV1 - Total Patients



| level_2_name  | level_2_id    | level_3_name  | level_3_id    | level_3_id   | source        |  periods      | value |
| ------------- | ------------- | ------------- | ------------- |------------- | ------------- | ------------- | ----- |


### Build the total number of patients for Cordaid

In [10]:
%%R
cordaid_exp <- subset(cordaid, dataElement==cordaid_id,  c(dataElement, period, orgUnit, categoryOptionCombo, attributeOptionCombo,value)  )
cordaid_full <- cordaid_exp %>% group_by(period, orgUnit) %>% summarize('value' = sum(value))
cordaid_full <- make_full_unique_set(cordaid_full)
adhoc_period <- data.frame('month' = unique(cordaid_full$periods),
                           'periods' = seq(201701,201711))
cordaid_full$source <- 'cordaid'
cordaid_imputed <- cordaid_full %>% group_by(orgUnit) %>% do(predict_missing(.))

|===================================================== | 99% ~0 s remaining     

### Build the total number of patients for PNLS

In [11]:
%%R
pnls_exp <- subset(pnls, dataElement==pnls_id,  c(dataElement, period, orgUnit, categoryOptionCombo, attributeOptionCombo,value)  )
pnls_full <- pnls_exp %>% group_by(period, orgUnit) %>% summarize('value' = sum(value))
pnls_full <- make_full_unique_set(pnls_full)
adhoc_period <- data.frame('month' = unique(pnls_full$periods),
                           'periods' = seq(201701,201711))
pnls_full$source <- 'pnls'
pnls_imputed <- pnls_full %>% group_by(orgUnit) %>% do(predict_missing(.))

|======================================================|100% ~0 s remaining     

### Build the total number of patients combined with both sources

In [12]:
%%R
## Making consolidation and imputation on set with both PNLS and CORDAID data
common_set <- build_common_set(cordaid_imputed, pnls_imputed, 'cordaid', 'pnls')
common_set$month <- period_monthly(common_set$periods)
completed_data_cordaid_pnls_total <- completed_data(common_set, 'cordaid', 'pnls')
adhoc_period <- data.frame('month' = unique(completed_data_cordaid_pnls_total$periods),
                           'periods' = seq(201701,201711))
imputed_data_cordaid_pnls_total <- completed_data_cordaid_pnls_total %>% group_by(orgUnit) %>% do(predict_missing(.))
imputed_data_cordaid_pnls_total <- imputed_data_cordaid_pnls_total[,colnames(pnls_imputed)]

In [168]:
%%R
## Keeping only data from CORDAID and PNLS with no value in other sources, and combining

imputed_data_cordaid_pnls_total$value_id <- paste0(imputed_data_cordaid_pnls_total$period, imputed_data_cordaid_pnls_total$orgUnit)
pnls_imputed$value_id <- paste0(pnls_imputed$period, pnls_imputed$orgUnit)
cordaid_imputed$value_id <- paste0(cordaid_imputed$period, cordaid_imputed$orgUnit)


pnls_unique <- subset(pnls_imputed, !(value_id %in% imputed_data_cordaid_pnls_total$value_id))
pnls_unique$source <- 'pnls'
cordaid_unique <- subset(cordaid_imputed, !(value_id %in% imputed_data_cordaid_pnls_total$value_id))
cordaid_unique$source <- 'cordaid'

imputed_data_cordaid_pnls_total$source <- 'combine'

col_keep <- c('periods', 'orgUnit', 'value', 'source')
final_imputation <- rbind(imputed_data_cordaid_pnls_total[,col_keep],pnls_unique[,col_keep],cordaid_unique[,col_keep])
final_imputation_hier <- merge(final_imputation, M_hierarchy, by.x = 'orgUnit', by.y = 'id')

### Output DS

In [169]:
%%R
ds1 <- final_imputation_hier %>% group_by(level_2_name, level_2_id, level_3_name, level_3_id, source, periods) %>% do(data.frame(value = sum(.$value)))
write.csv(ds1, 'df1.csv')

|===================================================== |100% ~0 s remaining     

## Treatment Lines Distribution

| level_2_name  | level_2_id    | level_3_name  | level_3_id    | level_3_id   |   periods      | value |
| ------------- | ------------- | ------------- | ------------- |------------- |  ------------- | ----- |


We look at this with two sourcses:
* PNLS - number of patients reported for each line
* CORDAID

We end up keeping the CORDAID data as it is the one with best completeness

In [132]:
%%R
drug_pnls_sex <- c('lttU7PiCAqg','prnCi6GwYzL','X6ROVclhfqF','X2nhX2N5B0B','WTfY6gHxkKe')
drug_pnls <- M_data_sets$DE_id[M_data_sets$DE_id %in% M_data_sets$DE_id[grep('DRUG' , M_data_sets$DE_name[M_data_sets$DE_id %in% pnls$dataElement])] & M_data_sets$categoryCombo.id =='kdfgXEYOVPb']
pnls_drugs <- pnls[pnls$dataElement %in% drug_pnls,]
pnls_drugs <- merge(pnls_drugs, M_data_sets[,c('DE_name', 'DE_id')], by.x = 'dataElement', by.y = 'DE_id', all.y = FALSE)
u <- pnls_drugs %>% group_by(dataElement, DE_name, parent.parent.parent.name,period) %>% do(data.frame(value = sum(.$value)))
out <- u %>% group_by(parent.parent.parent.name,period) %>% do(data.frame(DE_name = .$DE_name , value = .$value , share = .$value / sum(.$value)))

In [151]:
%%R
cc <- M_category_combos[M_category_combos$CatCombo_id == 'wnrmgFJSQ19',]
cordaid_traitements_id <- M_data_sets$DE_id[grep('\\+|(à préciser)' , 
                                                 M_data_sets$DE_name[M_data_sets$DE_id %in% cordaid$dataElement])]
cordaid_traitements <- subset(cordaid, dataElement %in% cordaid_traitements_id & 
                                           categoryOptionCombo %in% cc$CatComboOpt_id)
cordaid_traitements <- merge(cordaid_traitements, M_data_sets[,c('DE_name', 'DE_id')], by.x = 'dataElement', by.y = 'DE_id', all.y = FALSE)
u <- cordaid_traitements %>% group_by(dataElement, DE_name, parent.parent.name, parent.parent.id , parent.parent.parent.name, parent.parent.parent.id, period) %>% do(data.frame(value = sum(.$value)))
cordaid_out <- u %>% group_by(parent.parent.name, parent.parent.id , parent.parent.parent.name, parent.parent.parent.id, period) %>% do(data.frame(dataElement = .$dataElement, DE_name = .$DE_name , share = .$value / sum(.$value)))

|===================================================== |100% ~0 s remaining     

In [152]:
%%R
colnames(cordaid_out) <- c('level_3_name', 'level_3_id', 'level_2_name', 'level_2_id', 
                          'period', 'data_element_name', 'data_element_id', 'value')

In [153]:
%%R
write.csv(cordaid_out, 'df2.csv')

## Looking at stock outs

In [208]:
%%R
head(pnls_drugs)

  dataElement categoryOptionCombo     orgUnit     X period attributeOptionCombo
1 ALPM1l0eZ5c         ZrxeMshIRYT Gtpk8zxc3wD 20938 201707          HllvX50cXC0
2 ALPM1l0eZ5c         aNHHh7h8kX1 LM0iWoCnOSg 59549 201708          HllvX50cXC0
3 ALPM1l0eZ5c         XGYvPzLEgbM NzYYHRowJRo 21535 201711          HllvX50cXC0
4 ALPM1l0eZ5c         HKC0nz3BgDB CubYmLPFsDF 20281 201707          HllvX50cXC0
5 ALPM1l0eZ5c         XGYvPzLEgbM nITBxpUluRL 27217 201711          HllvX50cXC0
6 ALPM1l0eZ5c         bUjpYhZSGeN EvO1obd7JR9 11967 201710          HllvX50cXC0
  value storedBy                      created                  lastUpdated
1     0      d2d 2018-06-16T20:40:12.000+0000 2018-06-16T20:40:12.000+0000
2     0      d2d 2018-06-16T21:03:44.000+0000 2018-06-16T21:03:44.000+0000
3     0      d2d 2018-06-16T10:16:18.000+0000 2018-06-16T10:16:18.000+0000
4     0      d2d 2018-06-16T20:05:32.000+0000 2018-06-16T20:05:32.000+0000
5     0      d2d 2018-06-16T10:46:00.000+0000 2018-06-16T10:46:00

In [203]:
%%R 

exclude <- c('PNLS-DRUG-CTX 480 / 960 mg ces - Bt 500 ces', 'PNLS-DRUG-CTX 480 mg ces - Bt 1000 ces',
             'PNLS-DRUG-Hepatitis, HBsAg, Determine Kit, 100 Tests','PNLS-DRUG-Hepatitis, HCV, Rapid Device, Serum/Plasma/Whole Blood, kit de 40 Tests',
             'PNLS-DRUG-HIV 1/2, Double Check Gold, Kit de 100 test','PNLS-DRUG-HIV 1+2, Determine Complete, Kit de 100 tests',
             'PNLS-DRUG-HIV 1+2, Uni-Gold HIV, Kit de 20 tests', 'PNLS-DRUG-INH 100 mg; 300 mg - Cés', 'PNLS-DRUG-INH 50mg/5 ml - Sol.Orale',
             'PNLS-DRUG-Syphilis RPR Kit, kit de 100 Tests Determine syph')

In [205]:
%%R
drugs_pnls <- M_data_sets[M_data_sets$categoryCombo.id == 'Q3ONIkE9JN5' & !(M_data_sets$DE_name %in% exclude),]
pnls_drugs <- pnls[pnls$dataElement %in% drugs_pnls$DE_id, ]

pnls_drugs_cc <- merge(pnls_drugs, M_category_combos[,c('CatComboOpt_id', 'CatOpt_name.1')], by.x ='categoryOptionCombo', by.y = 'CatComboOpt_id'  )
pnls_drugs <- merge(pnls_drugs_cc, M_data_sets[,c('DE_name', 'DE_id')], by.x = 'dataElement', by.y = 'DE_id')

In [215]:
%%R
at_least_one_rs <- function(data){
    rs <- nrow(data[data$CatOpt_name.1 == 'Nbr de jours RS ' & data$value >0 ,]) > 1
    return(data.frame(rupture = rs))
}

fac_rs <- pnls_drugs %>% group_by(parent.parent.name, parent.parent.id , parent.parent.parent.name, parent.parent.parent.id, name ,period) %>% do(at_least_one_rs(.))

|===================================================== |100% ~0 s remaining     

In [218]:
%%R
perc_zone_rs <- fac_rs %>% group_by(parent.parent.name, parent.parent.id , parent.parent.parent.name, parent.parent.parent.id ,period) %>% do(data.frame(value = sum(.$rupture)/ nrow(.)))

In [223]:
%%R
write.csv(perc_zone_rs, 'df4.csv')

In [187]:
%%R
drugs <- c("PNLS-DRUG-AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces","PNLS-DRUG-AZT/3TC/NVP(300/150/200 mg)",
           "PNLS-DRUG-TDF/3TC/EFV(300/300/600 mg)","PNLS-DRUG-TDF/3TC/EFV(300/300/600 mg) - 30 ces")

pnls_drugs <- pnls_drugs[pnls_drugs$DE_name %in% drugs ,]
pnls_drugs$period <- as.Date(paste0(pnls_drugs$period, '28'), format = "%Y%m%d")

In [188]:
%%R
#Making CMM
conso <- pnls_drugs %>% filter(CatOpt_name.1 == 'Sortie ')

conso_rm <- conso %>% group_by(name, DE_name) %>% select(period, value) %>% mutate(cmm = rollmean(value, k = 3, align = 'left', fill=NA))


large <- dcast(pnls_drugs, name + DE_name + period ~ CatOpt_name.1, fun=mean)
colnames(large) <- gsub(' ', '_',trimws(colnames(large)))

large_cmm <- merge(large, conso_rm, all.x=TRUE , by = c('name','DE_name','period'))
large_cmm$alert[large_cmm$Stock_disponible_utilisable <= large_cmm$cmm] <- 'Insufficient Stocks'
large_cmm$alert[large_cmm$Stock_disponible_utilisable > large_cmm$cmm] <- 'Sufficient Stocks'
large_cmm$alert[is.na(large_cmm$alert)] <- 'Insufficient Info'
large_cmm$rupture[(large_cmm$Nbr_de_jours_RS > 0) & !is.na(large_cmm$Nbr_de_jours_RS)] <- large_cmm$period[large_cmm$Nbr_de_jours_RS > 0 & !is.na(large_cmm$Nbr_de_jours_RS)]


In [189]:
%%R 
head(large_cmm)

                                   name
1             bu Akpudu Centre de Santé
2             bu Akpudu Centre de Santé
3 bu Amadi Centre de Santé de Référence
4  bu Ango Hôpital Général de Référence
5  bu Ango Hôpital Général de Référence
6  bu Ango Hôpital Général de Référence
                                              DE_name     period Entrée
1 PNLS-DRUG-AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces 2017-01-28      0
2      PNLS-DRUG-TDF/3TC/EFV(300/300/600 mg) - 30 ces 2017-01-28      0
3      PNLS-DRUG-TDF/3TC/EFV(300/300/600 mg) - 30 ces 2017-10-28   6480
4 PNLS-DRUG-AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces 2017-07-28      0
5 PNLS-DRUG-AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces 2017-08-28      0
6 PNLS-DRUG-AZT/3TC/NVP 60/30/50 mg ces disp - 60 ces 2017-09-28      0
  Nbr_de_jours_RS Sortie Stock_disponible_utilisable Stock_Initial value cmm
1              31      0                           0             0     0  NA
2               0     60                         180          

In [191]:
%%R
## Imputing stock outs
tdf_ftc_efv <- c('TDF+3TC+EFV', 'PNLS-DRUG-TDF+3TC+EFV sex', 'PNLS-DRUG-TDF+3TC+EFV')
azt_3tc_nvp <- c('AZT+3TC+NVP', 'PNLS-DRUG-AZT+3TC+NVP', 'PNLS-DRUG-AZT/3TC/NVP(300/150/200 mg) - 60 ces')


tdf_ftc_efv_data <- pnls[pnls$dataElement %in% M_data_sets$DE_id[M_data_sets$DE_name %in% tdf_ftc_efv] ,]

tdf_ftc_efv_data$period <- as.Date(paste0(tdf_ftc_efv_data$period, '28'), format = "%Y%m%d")
patient_data <- tdf_ftc_efv_data %>% group_by(orgUnit, period) %>% summarise(n_patients = sum(value))

drug_data <- dcast(pnls_drugs, orgUnit + name + period ~ CatOpt_name.1, fun=mean)
colnames(drug_data) <- gsub(' ', '_',trimws(colnames(drug_data)))

full_data <- full_join(patient_data, drug_data, by= c('orgUnit','period'))

In [192]:
%%R

## Keep only places where we think we have number of boxes (and not cachetons)
make_ratio <- function(data){
  data <- data[!is.na(data$n_patients),]
  ratio <- median(data$Sortie/data$n_patients, na.rm = TRUE)
  return(ratio)
}

fosa_ratio <- full_data %>% group_by(orgUnit) %>% do(data.frame(ratio = make_ratio(.)))

data_test <- full_data[full_data$orgUnit %in% fosa_ratio$orgUnit[fosa_ratio$ratio < 2],]

|===================================================== |100% ~0 s remaining     

In [194]:
%%R

full_framework <- melt(dcast(data_test[,c('orgUnit','period')], orgUnit~period), variable.name = "period")

full_framework$period <- as.Date(full_framework$period)

data_test <- full_join(data_test, full_framework[,c('orgUnit','period')])


dat <- data_test[data_test$orgUnit == 'SIqwiCBwiL5',]


In [196]:
%%R
logisitics_path <- function(data, periods){
  cmm <- NA
  cmm_trace <- c()
  stock <- c()
  stock_comment <- c()
  sortie <- c()
  sortie_comment <- c()
  for( period in periods){
    period_data <- data[data$period == period,]
  ## Manage Sorties
    sortie_p <- period_data$Sortie
    sortie_c <- 'declared sortie'
    #if(is.na(sortie_p)){
    #  sortie_p <- period_data$n_patients
    #  sortie_c <- 'declared patients'
    #}
    if(is.na(sortie_p)){
      sortie_p <- cmm
      sortie_c <- 'CMM'
    }
    sortie <- c(sortie, sortie_p)
    sortie_comment <- c(sortie_comment, sortie_c)
    win_open <- max(1, length(sortie)-2)
    cmm <- mean(sortie[win_open:length(sortie)], na.rm=TRUE)
    cmm_trace <- c(cmm_trace, cmm)
  
  
    ## Manage Stock
    stock_p <- period_data$Stock_disponible_utilisable
    stock_i <- period_data$Stock_Initial
    stock_c <- 'declared_stock'
    if(is.na(stock_i)){
      stock_i <- stock[length(stock)]
      stock_c <- 'computed stock'
    }
    if(is.na(stock_p)){
      stock_p <- stock_i - sortie_p
      stock_c <- 'computed stock'
    }
    stock <- c(stock, max(0,stock_p))
    stock_comment <- c(stock_comment, stock_c)
  }
  out <- data.frame(stock, stock_comment, cmm_trace, sortie, sortie_comment, 'period'=periods)
  return(out)
}

data_test$Sortie[data_test$Sortie == 0] <- NA
data_test$Stock_disponible_utilisable[data_test$Stock_disponible_utilisable == 0] <- NA



out <- data_test %>% group_by(orgUnit) %>% do(logisitics_path(., sort(unique(data_test$period))))
out[out$stock_comment == 'computed stock' & !is.na(out$stock) & out$stock > 0,] %>% group_by(orgUnit) %>% do(data.frame(nrow(.)))

p <- out[out$stock_comment == 'computed stock' & !is.na(out$stock) & out$stock > 0 & out$sortie > 100,] %>% group_by(orgUnit) %>% do(data.frame(nrow(.)))


|===================================================== |100% ~0 s remaining     

In [199]:
%%R

out

# A tibble: 9,273 x 7
# Groups:   orgUnit [843]
   orgUnit     stock stock_comment  cmm_trace sortie sortie_comment  period    
   <fct>       <dbl> <chr>              <dbl>  <dbl> <chr>           <date>    
 1 BPpT4dTU7h9  51.5 declared_stock       6.5    6.5 declared sortie 2017-01-28
 2 BPpT4dTU7h9  31.5 declared_stock       7.5    8.5 declared sortie 2017-02-28
 3 BPpT4dTU7h9  24   computed stock       7.5    7.5 CMM             2017-03-28
 4 BPpT4dTU7h9  55   declared_stock      20.7   46   declared sortie 2017-04-28
 5 BPpT4dTU7h9  28   declared_stock      28.5   32   declared sortie 2017-05-28
 6 BPpT4dTU7h9   0   computed stock      35.5   28.5 CMM             2017-06-28
 7 BPpT4dTU7h9 106.  computed stock      31.8   35   declared sortie 2017-07-28
 8 BPpT4dTU7h9  72   computed stock      32.3   33.5 declared sortie 2017-08-28
 9 BPpT4dTU7h9  34.5 computed stock      35     36.5 declared sortie 2017-09-28
10 BPpT4dTU7h9  96.5 declared_stock      38.7   46   declared sortie 201